In [ ]:
import os
import polars as pl
import numpy as np
import torch.nn as nn
from polars import DataFrame
import torch
from torch.utils.data import DataLoader
from typing import Tuple
import wandb

from path import RESULT_DIRECTORY
from develop import reload_function, reload_module

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [ ]:
from data.source.pg_experiment import get_pg_experiment_dataframe

df_pronv1, _ = get_pg_experiment_dataframe(".ogg", assesment_version="v1")
df_pronv2, _ = get_pg_experiment_dataframe(".ogg", assesment_version="v2")

dataframe = df_pronv1.join(df_pronv2, on=["id_student", "word_id"], how="inner", suffix="_v2")
dataframe = dataframe.rename({"value": "v1_value", "value_v2": "v2_value"})
dataframe = dataframe.with_columns(word_id = pl.struct("word_id").rank("dense"))
dataframe = dataframe.filter(pl.col("stage") == 1)

df_outer = dataframe.filter(pl.col("v1_value") != pl.col("v2_value"))
df_inner = dataframe.filter(pl.col("v1_value") == pl.col("v2_value"))



N_WORDS = dataframe.select(pl.col("word_id").n_unique()).to_numpy()[0][0]
print(f"Number of unique words: {N_WORDS}")
print(f"Number of samples: {dataframe.shape[0]}")
print(f"Samples with v1_value != v2_value: {df_outer.shape[0]}")

In [ ]:
def split(df_outer: pl.DataFrame, df_inner: pl.DataFrame, label_col: str, train_frac=0.8, val_frac=0.1, seed=42) -> Tuple[pl.DataFrame, pl.DataFrame, pl.DataFrame]:
    classes = df_outer.select(label_col).unique().to_series()
    train_rows, val_rows, test_rows = [], [], []
    rng = np.random.RandomState(seed)
    df_sum = pl.concat([df_outer, df_inner])
    
    n_subset = df_inner.height
    n_test = int((1 - train_frac - val_frac) * df_sum.height)
    indices_subset = rng.permutation(n_subset)
    test_rows.append(df_inner[indices_subset[:n_test]])
    
    df = pl.concat([df_outer, df_inner[indices_subset[n_test:]]])
    
    # update fractions
    total_frac = train_frac + val_frac
    train_frac = train_frac / total_frac
    val_frac = val_frac / total_frac


    for cls in classes:
        class_df = df.filter(pl.col(label_col) == cls)
        n = class_df.height
        indices = rng.permutation(n)

        train_end = int(train_frac * n)
        val_end = int((train_frac + val_frac) * n)

        train_rows.append(class_df[indices[:train_end]])
        val_rows.append(class_df[indices[train_end:val_end]])

    train_df = pl.concat(train_rows)
    val_df = pl.concat(val_rows)
    test_df = pl.concat(test_rows)

    return train_df, val_df, test_df

In [ ]:
from dataset import Cast, TorchDataset
from pytorch_dataloader import build_collate_fn, PaddingCollate, DefaultCollate
from transformation import Channels, RMSEnergy, TorchVadMFCC, ZeroCrossingRate

TRAIN_SPLIT = 0.6
VAL_SPLIT = 0.2
TEST_SPLIT = 1 - TRAIN_SPLIT - VAL_SPLIT
train_pl, val_pl, test_pl = split(df_outer=df_outer, df_inner=df_inner, label_col="word_id", train_frac=TRAIN_SPLIT, val_frac=VAL_SPLIT)
val_pl = val_pl.filter(pl.col("v1_value") == pl.col("v2_value"))
test_pl = test_pl.filter(pl.col("v1_value") == pl.col("v2_value"))

def to_dataset(dataframe: DataFrame) -> TorchDataset:
    return TorchDataset(
    Cast(dataframe.get_column("rec_path"), Channels("stack","multiply")(
            TorchVadMFCC(delta=0),
        )),
    Cast(dataframe.get_column("rec_path"), Channels("cat","multiply")(
            ZeroCrossingRate(),
            RMSEnergy(),
        )),
    Cast(dataframe.get_column("word_id"), lambda x: torch.tensor(x-1, dtype=torch.long)),
    Cast(dataframe.get_column("v1_value"), lambda x: torch.tensor(x).float()),
    Cast(dataframe.get_column("v2_value"), lambda x: torch.tensor(x).float()),
)

collate_fn = build_collate_fn(
    PaddingCollate(mode="SET_MAX_LEN", max_len=80, pad_dim=2),
    PaddingCollate(mode="SET_MAX_LEN", max_len=80, pad_dim=1),
    DefaultCollate(),
    DefaultCollate(),
    DefaultCollate(),
)
dataset_train = to_dataset(train_pl)
dataset_val = to_dataset(val_pl)
dataset_test = to_dataset(test_pl)

In [ ]:
from os import name
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#note, if you are using Windows you MUST set `num_workers=0` - TL;DT multithreading DON'T work in notebooks because Windows DON'T have `fork()`
num_workers = 0 if name == "nt" else 4
train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True, collate_fn=collate_fn, num_workers=num_workers)
val_loader = DataLoader(dataset_val, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=num_workers)
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=num_workers)

for x in next(iter(train_loader)):
    print(x.shape)

In [ ]:
from pytorch_dataloader import MemoryLoadedDataLoader
train_loader = MemoryLoadedDataLoader(train_loader, device=device)
print("Loaded train loader into memory")
val_loader = MemoryLoadedDataLoader(val_loader, device=device)
print("Loaded validation loader into memory")
test_loader = MemoryLoadedDataLoader(test_loader, device=device)

In [ ]:
from models.FusionCNN import ContextFusionCNN, ContextCNN
from models.Guesser import ContextGuesser

cfcnn_model = ContextFusionCNN(n_2d_channels=1, n_1d_channels=2, num_words=N_WORDS)
ccnn_model = ContextCNN(n_2d_channels=1, num_words=N_WORDS)
guesser_model = ContextGuesser(num_words=N_WORDS)
models = [guesser_model, cfcnn_model, ccnn_model]

## Hyperparameters

In [ ]:
from type_helpers import Kwargs

epochs = 140

Optimizer = torch.optim.Adam
OptimizerKwargs = Kwargs(Optimizer)
optimizer_config = OptimizerKwargs(
    lr=1e-4,
    weight_decay= 1e-4,
)

Scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau
SchedulerKwargs = Kwargs(Scheduler)

scheduler_config = SchedulerKwargs(
    mode='min',
    factor=0.5,
    patience=5
)
criterion = nn.BCELoss()


In [ ]:
from models.SimpleCNN_v2 import train, evaluate

def train_model(model):
    model = model.to(device)
    optimizer = Optimizer(model.parameters(), **optimizer_config)
    scheduler = Scheduler(optimizer, **scheduler_config)

    run = wandb.init(
        # name of the run
        name=f"14+11 experiment - {model.__class__.__name__}",
        config={
            "Name": f"{model.__class__.__name__}",
            "learning_rate": optimizer_config["lr"],
            "optimizer": f"{Optimizer.__name__}",
            "criterion": f"{criterion.__class__.__name__}",
            "architecture": f"{model.__class__.__name__}",
            "architecture_details": str(model),
            "dataset": "Stage-I",
            "train_val_test(%)": f'{TRAIN_SPLIT}-{VAL_SPLIT}-{TEST_SPLIT}',
            "epochs": epochs,
        },
    )

    # Training loop
    for epoch in range(epochs):
        for label_version in ["v1","v2"]:
            train_loss, train_acc = train(model, train_loader, optimizer, criterion, device, label_version=label_version, interleave_labels=True)
            val_loss, val_acc = evaluate(model, val_loader, criterion, device, label_version=label_version, interleave_labels=True)
            scheduler.step(val_loss)
        run.log({"train_acc": train_acc, "train_loss": train_loss, "val_acc": val_acc, "val_loss": val_loss, })
        print(
            f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    run.log({"model_eval": model.eval()})
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    run.log({"test_acc": test_acc, "test_loss": test_loss})
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
    # Saving the model to pth and adding it to the artifacts of the run, there is 5GB of memory on wandb, so we should be fine.
    torch.save(model.state_dict(), os.path.join(RESULT_DIRECTORY, f"{model.__class__.__name__}.pth"))
    artifact = wandb.Artifact(f"{model.__class__.__name__}-model", type="model")
    artifact.add_file(os.path.join(RESULT_DIRECTORY, f"{model.__class__.__name__}.pth"))
    run.log_artifact(artifact)

    # Finish the run so it gets sent to the remote. You can discover the run right after that on the dashboard.
    run.finish()


In [ ]:
for model in models:
    train_model(model)